In [5]:
import gym
import minerl

C:\Users\georg\anaconda3\envs\minerl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
%run parse_missions

In [7]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
env = gym.make("MineRLNavigate-v0", xml="my_mission_xmls/navigationFixedLTH.xml")

INFO:minerl.env.malmo.instance.e00dde:Starting Minecraft process: ['C:\\Users\\georg\\AppData\\Local\\Temp\\tmpv9o50wxi\\Minecraft\\launchClient.bat', '-port', '11913', '-env', '-runDir', 'C:\\Users\\georg\\AppData\\Local\\Temp\\tmpv9o50wxi\\Minecraft\\run']
INFO:minerl.env.malmo.instance.e00dde:Starting process watcher for process 8968 @ localhost:11913


In [5]:
obs = env.reset()

DEBUG:minerl.env.core:Creating socket connection!
DEBUG:minerl.env.core:Sending mission init!
DEBUG:minerl.env.malmo.instance.37f227:[12:09:29] [EnvServerSocketHandler/INFO]: [STDOUT]: Received from 127.0.0.1:<MissionInit xmlns="http://ProjectMalmo.microsoft.com" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" SchemaVersion="" PlatformVersion="0.37.0">
DEBUG:minerl.env.malmo.instance.37f227:                                <Mission>
DEBUG:minerl.env.malmo.instance.37f227:    <About>
DEBUG:minerl.env.malmo.instance.37f227:        <Summary>MineRLNavigate-v0</Summary>
DEBUG:minerl.env.malmo.instance.37f227:    </About>
DEBUG:minerl.env.malmo.instance.37f227:
DEBUG:minerl.env.malmo.instance.37f227:    <ModSettings>
DEBUG:minerl.env.malmo.instance.37f227:        <MsPerTick>50</MsPerTick>
DEBUG:minerl.env.malmo.instance.37f227:    </ModSettings>
DEBUG:minerl.env.malmo.instance.37f227:
DEBUG:minerl.env.malmo.instance.37f227:    <ServerSection>
DEBUG:minerl.env.malmo.instance.37f227:     

DEBUG:minerl.env.malmo.instance.37f227:[12:09:35] [Client thread/INFO]: [STDOUT]: Resized the window
DEBUG:minerl.env.malmo.instance.37f227:[12:09:35] [Server thread/INFO]: [STDOUT]: SERVER request state: RUNNING
DEBUG:minerl.env.malmo.instance.37f227:[12:09:35] [Client thread/INFO]: [STDOUT]: Unpausing
DEBUG:minerl.env.malmo.instance.37f227:[12:09:35] [Server thread/INFO]: [STDOUT]: SERVER enter state: RUNNING
DEBUG:minerl.env.malmo.instance.37f227:[12:09:36] [Client thread/INFO]: [CHAT] §l300...


In [6]:
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, done, _ = env.step(action)

DEBUG:minerl.env.malmo.instance.37f227:[12:09:45] [Server thread/INFO]: [STDOUT]: Warning: managed 3/20 ticks this second.
DEBUG:minerl.env.malmo.instance.37f227:[12:09:45] [Client thread/INFO]: [CHAT] §l299...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:46] [Client thread/INFO]: [CHAT] §l298...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:46] [Client thread/INFO]: [CHAT] §l297...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:46] [Client thread/INFO]: [CHAT] §l296...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Client thread/INFO]: [CHAT] §l295...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Client thread/INFO]: [CHAT] §l294...
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Client thread/INFO]: Stopping!
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Client thread/INFO]: [STDOUT]: Unpausing
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Server thread/INFO]: Stopping server
DEBUG:minerl.env.malmo.instance.37f227:[12:09:47] [Server thread/INFO]: Saving players


In [7]:
env.close()

ERROR:minerl.env.malmo:Attempted to send kill command to minecraft process and failed.
